1.데이터수집

In [7]:
import pandas as pd
df = pd.read_csv('day5_data1.csv')
df

,who,Newbie,Age,Gender,Household Income,Sexual Preference,Country,Education Attainment,Major Occupation,Marital Status,Years on Internet
0,id74364,0,54.0,Male,$50-74,Gay male,Ontario,Some College,Computer,Other,4-6 yr
1,id84505,0,39.0,Female,Over $100,Heterosexual,Sweden,Professional,Other,Other,1-3 yr
2,id84509,1,49.0,Female,$40-49,Heterosexual,Washington,Some College,Management,Other,Under 6 mo
3,id87028,1,22.0,Female,$40-49,Heterosexual,Florida,Some College,Computer,Married,6-12 mo
4,id76087,0,20.0,Male,$30-39,Bisexual,New Jersey,Some College,Education,Single,1-3 yr
...,...,...,...,...,...,...,...,...,...,...,...
19578,id83400,0,22.0,Male,Over $100,Heterosexual,Texas,Some College,Education,Single,4-6 yr
19579,id72216,0,19.0,Male,NaN,Heterosexual,New Jersey,Some College,Education,Single,4-6 yr
19580,id8654,0,49.0,Female,$50-74,Heterosexual,Missouri,Doctoral,Education,Married,1-3 yr
19581,id84503,1,42.0,Female,$50-74,Heterosexual,Kentucky,Some College,Other,Married,Under 6 mo


2. data 전처리작업<br> 1) '필요 유무에따른 정리'

In [8]:
df.pop('who')
df.pop('Country')
df.pop('Years on Internet')
#제거 했으니 검토해야한다.
df.dtypes

Newbie                    int64
Age                     float64
Gender                   object
Household Income         object
Sexual Preference        object
Education Attainment     object
Major Occupation         object
Marital Status           object
dtype: object

In [11]:
#범주데이터를 참거짓으로 표현해준다.
#object class를 범주화 작업
ck_c=['Gender','Household Income','Sexual Preference','Education Attainment','Major Occupation','Marital Status']
for i in ck_c:
    df[i] = df[i].astype('category') #시리즈마다 따로처리하기위해 사용/ category로 바꿔준다
df.dtypes

Newbie                     int64
Age                      float64
Gender                  category
Household Income        category
Sexual Preference       category
Education Attainment    category
Major Occupation        category
Marital Status          category
dtype: object

2.data 전처리 <br> 2)결측치

In [19]:
#범주화된 데이텉를 원핫인코딩화 해줌 
df_one_hot=pd.get_dummies(df)
df_one_hot#각각의 열들은 데이터 인스턴스에스 특징을 의미한다./ 즉 데이터의 특징을 2가지로 나눠줬다.
df_one_hot.isnull().sum()

Newbie                                 0
Age                                  561
Gender_Female                          0
Gender_Male                            0
Household Income_$10-19                0
Household Income_$20-29                0
Household Income_$30-39                0
Household Income_$40-49                0
Household Income_$50-74                0
Household Income_$75-99                0
Household Income_Over $100             0
Household Income_Under $10             0
Sexual Preference_Bisexual             0
Sexual Preference_Gay male             0
Sexual Preference_Heterosexual         0
Sexual Preference_Lesbian              0
Sexual Preference_Transgender          0
Sexual Preference_na                   0
Education Attainment_College           0
Education Attainment_Doctoral          0
Education Attainment_Grammar           0
Education Attainment_High School       0
Education Attainment_Masters           0
Education Attainment_Other             0
Education Attain

In [20]:
df_one_hot.loc[pd.isnull(df_one_hot['Age']),'Age']=df_one_hot['Age'].mean() #결측치가 존재하는 age인덱스로 접근해줌

3.입력data정리

In [27]:
X = df_one_hot.iloc[:,1:].values
Y = df_one_hot.iloc[:,0].values.reshape(-1,1)#단일데이터화 시켜줌
X.shape, Y.shape # 동일한 사이즈로 변환해주는것이 정규화작업

((19583, 37), (19583, 1))

In [28]:
from sklearn.preprocessing import MinMaxScaler
m_m_s=MinMaxScaler()
#전체데이터를 다룰때
X_data=m_m_s.fit_transform(X)#정규화 작업 스케일작업은 너무 쏠려있으면 나쁜상태라 평균작업을해준것

In [29]:
from sklearn.model_selection import train_test_split
t_x,tt_x,t_y,tt_y = train_test_split(X_data,Y,test_size=0.3,random_state=42)

4. 모델 생성 및 학습

In [32]:
t_y.flatten()#전환 작업

array([1, 1, 0, ..., 1, 0, 1], dtype=int64)

In [33]:
from sklearn.linear_model import LogisticRegression
lo_g=LogisticRegression(fit_intercept=True)#설정해둔 목표값에 도달하면 값에대한 것을 판단할수있다.
lo_g.fit(t_x,t_y.flatten())


LogisticRegression()

5.테스트 및 검증

In [34]:
lo_g.predict(tt_x[:5])

array([0, 0, 0, 0, 0], dtype=int64)

In [35]:
tt_y.flatten()[:5]

array([1, 0, 1, 0, 1], dtype=int64)

In [36]:
lo_g.predict_proba(tt_x[:5])

array([[0.56799173, 0.43200827],
       [0.91026881, 0.08973119],
       [0.79110791, 0.20889209],
       [0.85449667, 0.14550333],
       [0.62307651, 0.37692349]])

In [37]:
from sklearn.metrics import confusion_matrix#매트릭스
from sklearn.metrics import accuracy_score#정확도
y_t = tt_y.copy()
y_p = lo_g.predict(tt_x)
confusion_matrix(y_t,y_p)

array([[4093,  240],
       [1221,  321]], dtype=int64)

In [38]:
accuracy_score(y_t,y_p)#75% 정확도 나옴

0.7513191489361702